In [1]:
import pandas as pd
from tqdm import tqdm
from sqlalchemy import create_engine, inspect
tqdm.pandas(desc="Processing")

In [2]:
%reload_ext sql
%sql mysql://iasamori@localhost/?charset=utf8mb4
%config SqlMagic.displaylimit = 1000
%sql USE iasamori;

There's a new jupysql version available (0.11.1), you're running 0.10.9. To upgrade: pip install jupysql --upgrade
Deploy Streamlit apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


Connecting to 'mysql://iasamori@localhost/?charset=utf8mb4'

Running query in 'mysql://iasamori@localhost/?charset=utf8mb4'

++
||
++
++

In [3]:
%load_ext rpy2.ipython

In [4]:
%%R

source("../psych290_data/helper_files/psych290RcodeV1.R")

require(tidyverse)
require(RSQLite)
require(ggthemes)
require(ggpubr)
require(grid)
require(reshape2)
require(psych)
require(lm.beta)
library(caret)

library(DBI)
library(RMySQL)

dbcon <- dbConnect(
  drv = MySQL(),
  host = "localhost",  # or your server address
  dbname = "iasamori",
  user = "iasamori",
  port = 3306
)

R[write to console]: Loading required package: ggthemes

R[write to console]: Loading required package: tidyverse



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


R[write to console]: Loading required package: RSQLite

R[write to console]: Loading required package: ggpubr

R[write to console]: Loading required package: psych

R[write to console]: 
Attaching package: ‘psych’


R[write to console]: The following objects are masked from ‘package:ggplot2’:

    %+%, alpha


R[write to console]: Loading required package: lm.beta

R[write to console]: Loading required package: lattice

R[write to console]: 
Attaching package: ‘caret’


R[write to console]: The following object is masked from ‘package:purrr’:

    lift




In [5]:
engine = create_engine(f"mysql://iasamori@localhost/iasamori?charset=utf8mb4")
meta_tcp7_my = pd.read_sql_query("SELECT * FROM meta_tcp7_my", engine)
engine.dispose()

In [6]:
meta_tcp7_my["month"] = [int(i) for i in meta_tcp7_my["group_id"].str.split("_", expand=True)[0]]
meta_tcp7_my["month_str"] = meta_tcp7_my["month"].astype(str).str.zfill(2)
meta_tcp7_my["year"] = [int(i) for i in meta_tcp7_my["group_id"].str.split("_", expand=True)[1]]
meta_tcp7_my["Month Code"] = meta_tcp7_my["year"].astype(str) + "/" + meta_tcp7_my["month_str"]
meta_tcp7_my = meta_tcp7_my.sort_values(by=["year", "month"])
meta_tcp7_my = meta_tcp7_my.reset_index(drop=True)
meta_tcp7_my

,group_id,_avg1gramLength,_avg1gramsPerMsg,_total1grams,_totalMsgs,_intercept,0,1,10,100,...,94,95,96,97,98,99,month,month_str,year,Month Code
0,12_2020,4.166667,6.000000,6.0,1.0,1.0,0.000000,0.001587,0.000000,0.000000,...,0.000939,0.000000,0.000000,0.002057,0.008617,0.000000,12,12,2020,2020/12
1,1_2021,3.563996,18.006653,27064.0,1503.0,1.0,0.001447,0.001786,0.001284,0.001770,...,0.003066,0.001856,0.001837,0.002593,0.002081,0.001056,1,01,2021,2021/01
2,2_2021,3.688290,16.068335,16695.0,1039.0,1.0,0.002392,0.001710,0.001697,0.001766,...,0.002365,0.001619,0.001724,0.003770,0.002006,0.001492,2,02,2021,2021/02
3,3_2021,3.697761,18.284120,21301.0,1165.0,1.0,0.001575,0.001816,0.001321,0.002326,...,0.003712,0.001946,0.001295,0.001546,0.002030,0.001485,3,03,2021,2021/03
4,4_2021,3.542862,19.072859,55235.0,2896.0,1.0,0.001103,0.001427,0.001174,0.001730,...,0.002221,0.001559,0.001073,0.001579,0.002358,0.001332,4,04,2021,2021/04
5,5_2021,3.592880,19.436309,71409.0,3674.0,1.0,0.000923,0.001540,0.001151,0.001752,...,0.002225,0.001696,0.001145,0.001482,0.002016,0.001429,5,05,2021,2021/05
6,6_2021,3.770982,18.540146,43180.0,2329.0,1.0,0.001821,0.002116,0.001634,0.002161,...,0.002322,0.001919,0.001413,0.001777,0.001754,0.001465,6,06,2021,2021/06
7,7_2021,3.739453,16.362348,8083.0,494.0,1.0,0.001596,0.002081,0.000926,0.001691,...,0.002823,0.002589,0.016589,0.002204,0.001415,0.001546,7,07,2021,2021/07
8,8_2021,3.648688,19.600000,2058.0,105.0,1.0,0.001055,0.001847,0.001609,0.002242,...,0.002936,0.002005,0.001406,0.002301,0.002971,0.001140,8,08,2021,2021/08
9,9_2021,3.614741,18.019608,34922.0,1938.0,1.0,0.001264,0.002044,0.001282,0.002044,...,0.002845,0.001687,0.001419,0.001955,0.002209,0.001529,9,09,2021,2021/09


In [7]:
tcp7_my = meta_tcp7_my.drop(["_avg1gramLength",	"_avg1gramsPerMsg",	"_total1grams",	"_totalMsgs",	"_intercept"], axis=1)
tcp7_my

,group_id,0,1,10,100,101,102,103,104,105,...,94,95,96,97,98,99,month,month_str,year,Month Code
0,12_2020,0.000000,0.001587,0.000000,0.000000,0.000000,0.003819,0.000000,0.000000,0.000972,...,0.000939,0.000000,0.000000,0.002057,0.008617,0.000000,12,12,2020,2020/12
1,1_2021,0.001447,0.001786,0.001284,0.001770,0.001845,0.001761,0.002165,0.001946,0.003465,...,0.003066,0.001856,0.001837,0.002593,0.002081,0.001056,1,01,2021,2021/01
2,2_2021,0.002392,0.001710,0.001697,0.001766,0.001844,0.001751,0.002028,0.001719,0.002499,...,0.002365,0.001619,0.001724,0.003770,0.002006,0.001492,2,02,2021,2021/02
3,3_2021,0.001575,0.001816,0.001321,0.002326,0.002201,0.002283,0.003623,0.001679,0.007434,...,0.003712,0.001946,0.001295,0.001546,0.002030,0.001485,3,03,2021,2021/03
4,4_2021,0.001103,0.001427,0.001174,0.001730,0.002669,0.002520,0.003153,0.002088,0.002516,...,0.002221,0.001559,0.001073,0.001579,0.002358,0.001332,4,04,2021,2021/04
5,5_2021,0.000923,0.001540,0.001151,0.001752,0.002691,0.003066,0.002517,0.002172,0.002370,...,0.002225,0.001696,0.001145,0.001482,0.002016,0.001429,5,05,2021,2021/05
6,6_2021,0.001821,0.002116,0.001634,0.002161,0.001928,0.001814,0.002804,0.001946,0.003266,...,0.002322,0.001919,0.001413,0.001777,0.001754,0.001465,6,06,2021,2021/06
7,7_2021,0.001596,0.002081,0.000926,0.001691,0.001567,0.002017,0.001750,0.001897,0.002855,...,0.002823,0.002589,0.016589,0.002204,0.001415,0.001546,7,07,2021,2021/07
8,8_2021,0.001055,0.001847,0.001609,0.002242,0.002498,0.001208,0.001896,0.002098,0.004109,...,0.002936,0.002005,0.001406,0.002301,0.002971,0.001140,8,08,2021,2021/08
9,9_2021,0.001264,0.002044,0.001282,0.002044,0.002532,0.002875,0.002612,0.002194,0.003903,...,0.002845,0.001687,0.001419,0.001955,0.002209,0.001529,9,09,2021,2021/09


In [8]:
tcp7_my.to_csv("./data/topic_norms2.csv", index=False)